In [1]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import os, sys, gc, time, warnings, pickle, random

from tsfmeta.data import MetaDataset,RawData, Md_utils,temporal_signal_split
from tsfmeta import utils
from tsfmeta import nn as MetaNN
from tsfmeta.experiments import meta_learning_run
from tqdm import tqdm

In [2]:

dir_ = 'E:/Datasets/KDD2022/' # input only here
sdwpf = pd.read_csv(dir_+'sdwpf_baidukddcup2022_full.CSV')
sdwpf


,TurbID,Day,Tmstamp,Wspd,Wdir,Etmp,Itmp,Ndir,Pab1,Pab2,Pab3,Prtv,Patv
0,1,1,00:00,12.23,-0.83,29.08,41.90,-23.73,1.07,1.07,1.07,-0.21,1549.53
1,1,1,00:10,11.58,-3.32,29.01,42.01,-23.70,1.06,1.06,1.06,-0.25,1549.71
2,1,1,00:20,11.21,-1.38,29.17,42.24,-28.84,1.04,1.04,1.04,-0.25,1534.77
3,1,1,00:30,10.84,0.06,29.46,42.43,-31.39,1.03,1.03,1.03,-0.25,1508.20
4,1,1,00:40,11.03,2.03,29.82,42.77,-31.39,1.03,1.03,1.03,-66.01,1517.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3550459,134,184,23:10,2.36,-74.19,7.30,11.70,238.59,90.39,90.37,90.36,-0.30,-0.30
3550460,134,184,23:20,1.72,-67.92,7.21,11.70,238.59,90.39,90.37,90.36,-0.30,-0.30
3550461,134,184,23:30,1.46,-59.15,7.10,11.70,238.59,90.39,90.37,90.36,-0.30,-0.30
3550462,134,184,23:40,1.31,-64.11,7.10,11.70,238.59,90.39,90.37,90.36,-0.30,-0.30


In [3]:
location = pd.read_csv(dir_+'sdwpf_baidukddcup2022_turb_location.CSV')
location_max = location[['x','y']].max()
location[['x','y']] = location[['x','y']]/location_max

In [4]:

sdwpf.loc[sdwpf['Patv'] < 0, 'Patv'] = 0
sdwpf = sdwpf.groupby(['TurbID','Day']).mean().reset_index()
sdwpf = sdwpf.merge(location,on = 'TurbID')
#sdwpf['TurbID_feature'] =  sdwpf['TurbID']  

In [5]:
target =['Patv']
features_num = ['Wspd','Wdir','Etmp','Itmp','Pab1','Pab2','Pab3','Prtv','x','y'] ## target should be the first
features_cat = [] 
idx = ['TurbID']  
tdx = ['Day']
dynamic_known_features_num = ['Pab1','Pab2','Pab3']  ## should be part of features_num
dynamic_known_features_cat = []   ## should be part of features_cat
static_known_features = ['x','y'] ## should be part of known_features_cat

df = Md_utils.Df_to_rawdata(df = sdwpf,
                   idx = idx,
                   tdx = tdx,
                   target = target,
                   features_num = features_num,
                   features_cat= features_cat,
                   static_known_features = static_known_features,
                   dynamic_known_features_num = dynamic_known_features_num,
                   dynamic_known_features_cat = dynamic_known_features_cat,
                   )

In [6]:
df_slots = df.data_segment(W=100, H=7,step=2,nseg = 40)
df_slots[3].npdata.shape
df_slots[0].time_idx

Int64Index([ 78,  79,  80,  81,  82,  83,  84,  85,  86,  87,
            ...
            175, 176, 177, 178, 179, 180, 181, 182, 183, 184],
           dtype='int64', name='Day', length=107)

In [7]:
for i in range(len(df_slots)):
    df_slots[i] = df_slots[i].Raw2Meta(H=7)
    #df_slots[i].reduce_mem_usage()

In [8]:
# SES example
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from random import random

for slot in range(len(df_slots)):

    Nts,_, T = df_slots[slot].X.shape 
    _, H = df_slots[slot].Y.shape
    model = [SimpleExpSmoothing(df_slots[slot].X[i,0]).fit() for i in range(Nts)]
    model_pred = [model[i].predict(T,T + H -1) for i in range(Nts)]
    model_pred = np.stack(model_pred,0)
    df_slots[slot].add_Base_forecasts(model_pred,'SimpleExpSmoothing')



c:\Users\MSH\.conda\envs\pytorch\lib\site-packages\statsmodels\tsa\holtwinters\model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning,
c:\Users\MSH\.conda\envs\pytorch\lib\site-packages\statsmodels\tsa\holtwinters\model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning,
c:\Users\MSH\.conda\envs\pytorch\lib\site-packages\statsmodels\tsa\holtwinters\model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning,
c:\Users\MSH\.conda\envs\pytorch\lib\site-packages\statsmodels\tsa\holtwinters\model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning,
c:\Users\MSH\.conda\envs\pytorch\lib\site-packages\statsmodels\tsa\holtwinters\model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning,
c:\Users\MSH\.conda\envs\pytorch\lib\site-packages\statsmodels\ts

In [9]:
# HEWS example
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from random import random

for slot in range(len(df_slots)):

    Nts,_, T = df_slots[slot].X.shape 
    _, H = df_slots[slot].Y.shape
    model = [ExponentialSmoothing(df_slots[slot].X[i,0], trend = 'add',damped_trend=True).fit() for i in range(Nts)]
    model_pred = [model[i].predict(T,T + H -1) for i in range(Nts)]
    model_pred = np.stack(model_pred,0)
    df_slots[slot].add_Base_forecasts(model_pred,'ExponentialSmoothing')

c:\Users\MSH\.conda\envs\pytorch\lib\site-packages\statsmodels\tsa\holtwinters\model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning,
c:\Users\MSH\.conda\envs\pytorch\lib\site-packages\statsmodels\tsa\holtwinters\model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning,
c:\Users\MSH\.conda\envs\pytorch\lib\site-packages\statsmodels\tsa\holtwinters\model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning,
c:\Users\MSH\.conda\envs\pytorch\lib\site-packages\statsmodels\tsa\holtwinters\model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning,
c:\Users\MSH\.conda\envs\pytorch\lib\site-packages\statsmodels\tsa\holtwinters\model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning,
c:\Users\MSH\.conda\envs\pytorch\lib\site-packages\statsmodels\ts

In [10]:
#lgb direct example
import lightgbm as lgb
from sklearn.model_selection import train_test_split

for slot in range(len(df_slots)):
    print(slot)
    gdf = df_slots[slot].XZ_globalReg(L=7, rolling_step = 1, mode= 'Tree')

    H = gdf['Y_tr'].shape[-1]
    X_train, X_val, y_train, y_val, mask_train, mask_val,  =  train_test_split(np.concatenate([gdf['X_tr'],gdf['Z_tr']],-1) ,gdf['Y_tr'] ,gdf['mask_tr'], test_size = 0.2, random_state =0)

    predicts = []
    for h in range(H): 
        dtrain = lgb.Dataset(X_train , label= y_train[:,h] ,weight = mask_train[:,h] )
        dval = lgb.Dataset(X_val , label= y_val[:,h] , weight = mask_val[:,h] )
        params = {
                'num_leaves': 128,
                'objective': 'regression',
                'min_data_in_leaf': 10,
                'learning_rate': 0.02,
                'feature_fraction': 0.8,
                'bagging_fraction': 0.7,
                'bagging_freq': 1,
                'metric': 'rmse',
                'num_threads': 8
            }
            
            
        MAX_ROUNDS = 2000
        bst = lgb.train(
                params, dtrain, num_boost_round=MAX_ROUNDS, 
                        valid_sets=[dtrain,dval], early_stopping_rounds=125, verbose_eval=30
                    )

        predicts.append( bst.predict(np.concatenate([gdf['X_ts'],gdf['Z_ts']],-1) , num_iteration=bst.best_iteration or MAX_ROUNDS)[:,np.newaxis])

    predicts = np.concatenate(predicts,-1)
    df_slots[slot].add_Base_forecasts(predicts,'lightgbm_direct')


0
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003407 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 20021
[LightGBM] [Info] Number of data points in the train set: 9219, number of used features: 79
[LightGBM] [Info] Start training from score 368.883862
Training until validation scores don't improve for 125 rounds


c:\Users\MSH\.conda\envs\pytorch\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\MSH\.conda\envs\pytorch\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[30]	training's rmse: 174.179	valid_1's rmse: 180.539
[60]	training's rmse: 104.966	valid_1's rmse: 115.274
[90]	training's rmse: 67.5316	valid_1's rmse: 82.2701
[120]	training's rmse: 47.4168	valid_1's rmse: 66.4001
[150]	training's rmse: 36.6831	valid_1's rmse: 59.2611
[180]	training's rmse: 30.4944	valid_1's rmse: 55.8302
[210]	training's rmse: 26.4782	valid_1's rmse: 53.8617
[240]	training's rmse: 23.5426	valid_1's rmse: 52.6896
[270]	training's rmse: 21.2478	valid_1's rmse: 51.8422
[300]	training's rmse: 19.357	valid_1's rmse: 51.2918
[330]	training's rmse: 17.744	valid_1's rmse: 50.8816
[360]	training's rmse: 16.3551	valid_1's rmse: 50.5994
[390]	training's rmse: 15.1212	valid_1's rmse: 50.3869
[420]	training's rmse: 14.0062	valid_1's rmse: 50.218
[450]	training's rmse: 13.0135	valid_1's rmse: 50.064
[480]	training's rmse: 12.1273	valid_1's rmse: 49.9593
[510]	training's rmse: 11.3093	valid_1's rmse: 49.8426
[540]	training's rmse: 10.5645	valid_1's rmse: 49.7703
[570]	training's 

In [11]:
#lgb mimo example
import lightgbm as lgb
from sklearn.model_selection import train_test_split

for slot in range(len(df_slots)):
    print(slot)
    gdf = df_slots[slot].XZ_globalReg(L=14, rolling_step = 1, mode= 'Tree')
    
    X = gdf['X_tr'].repeat(H,0)
    Y =  gdf['Y_tr'].reshape(-1)
    Z = np.stack([gdf['Z_tr'][:,(i*H):(i+1)*H].reshape(-1) for i in range(gdf['Z_tr'].shape[1]//H)],1)
    mask = gdf['mask_tr'].reshape(-1)

    H = gdf['Y_tr'].shape[-1]
    X_train, X_val, y_train, y_val, mask_train, mask_val,  =  train_test_split(np.concatenate([X,Z],-1) ,Y ,mask, test_size = 0.2, random_state =0)

 
    X_test = gdf['X_ts'].repeat(H,0)
    Z_test = np.stack([gdf['Z_ts'][:,(i*H):(i+1)*H].reshape(-1) for i in range(gdf['Z_ts'].shape[1]//H)],1)
    mask_val = mask_val.reshape(-1)
    
    dtrain = lgb.Dataset(X_train , label= y_train ,weight = mask_train )
    dval = lgb.Dataset(X_val , label= y_val, weight = mask_val )
    params = {
            'num_leaves': 128,
            'objective': 'regression',
            'min_data_in_leaf': 15,
            'learning_rate': 0.02,
            'feature_fraction': 0.8,
            'bagging_fraction': 0.7,
            'bagging_freq': 1,
            'metric': 'rmse',
            'num_threads': 8
        }
        
        
    MAX_ROUNDS = 2000
    bst = lgb.train(
            params, dtrain, num_boost_round=MAX_ROUNDS, 
                    valid_sets=[dtrain,dval], early_stopping_rounds=125, verbose_eval=30
                )

    predicts = bst.predict( np.concatenate([X_test,Z_test],-1), num_iteration=bst.best_iteration or MAX_ROUNDS)

    predicts = predicts.reshape(len(gdf['X_ts']),H )
    df_slots[slot].add_Base_forecasts(predicts,'lightgbm_mimo')


0
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.019453 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 29705
[LightGBM] [Info] Number of data points in the train set: 59281, number of used features: 117
[LightGBM] [Info] Start training from score 355.410227
Training until validation scores don't improve for 125 rounds
[30]	training's rmse: 211.817	valid_1's rmse: 217.189
[60]	training's rmse: 172.707	valid_1's rmse: 180.912
[90]	training's rmse: 152.534	valid_1's rmse: 163.45
[120]	training's rmse: 140.769	valid_1's rmse: 154.22
[150]	training's rmse: 132.858	valid_1's rmse: 148.592
[180]	training's rmse: 127.068	valid_1's rmse: 144.944
[210]	training's rmse: 122.473	valid_1's rmse: 142.363
[240]	training's rmse: 118.633	valid_1's rmse: 140.231
[270]	training's rmse: 115.219	valid_1's rmse: 138.351
[300]	training's rmse: 112.471	valid_1's rmse: 136.992
[330]	training's rmse: 110.296	valid_1's r

In [12]:
# random forest example
from sklearn.ensemble import RandomForestRegressor

for slot in range(len(df_slots)):
    print(slot)
    gdf = df_slots[slot].XZ_globalReg(L=7, rolling_step = 1, mode= 'Tree')

    H = gdf['Y_tr'].shape[-1]
        
    regr = RandomForestRegressor(n_estimators=200 ,max_depth=10, random_state=0, n_jobs = 4 )
    regr.fit(X = np.nan_to_num(np.concatenate([gdf['X_tr'],gdf['Z_tr']],-1),nan = -1),
             y = np.nan_to_num(gdf['Y_tr'] ,nan = -1),
             sample_weight = gdf['mask_tr'][:,0])    
    predicts = regr.predict(X = np.nan_to_num(np.concatenate([gdf['X_ts'],gdf['Z_ts']],-1),nan = -1))   
    df_slots[slot].add_Base_forecasts(predicts,'rf_mimo')
pickle.dump(df_slots, open(dir_ +'kdd2022_metadata_slots_v2' +'.pkl', 'wb'), protocol= 4)



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38


In [14]:
dir_

'E:/Datasets/KDD2022/'

In [15]:
pickle.dump(df_slots, open(dir_ +'kdd2022_metadata_slots_exp_h7_s2' +'.pkl', 'wb'), protocol= 4)

In [3]:
df_slots = pickle.load( open(dir_ +'kdd2022_metadata_slots_exp_h7_s2' +'.pkl', 'rb'))

In [4]:
### meta learning

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [5]:
learning_rate = 1e-5
batch_size = 256

meta_train = Md_utils.md_concat([df_slots[i] for i in range(8,39)],fidx = [0,1,2,3,4])

meta_test = Md_utils.md_concat([df_slots[i] for i in range(5)],fidx = [0,1,2,3,4])

train_dataloader = meta_train.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
test_dataloader = meta_test.to_dataloader(train=False, batch_size=batch_size, num_workers=0)

In [6]:
## metacomb example
preds = []
n_features = meta_train.Z.shape[1]
_, n_forecasters, horizon  = meta_train.B.shape
window_x = meta_train.X.shape[2]

n_known_features = len(meta_train.known_features)
Use_z = True

model = MetaNN.MetaComb(n_features= n_features, n_forecasters=n_forecasters,window_x=window_x,Use_z= Use_z,n_known_features= n_known_features, horizon=horizon).to(device)
loss_fn = torch.nn.MSELoss()
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(),  lr=learning_rate)
epochs = 50
meta_learning = meta_learning_run(model,loss_fn, optimizer, train_dataloader,test_dataloader, device, epochs)
loss, pred = meta_learning.forecast(test_dataloader)
preds.append(np.expand_dims(pred.cpu().numpy(),1))

Epoch 1
-------------------------------
66761.04541015625
42185.833333333336
Epoch 2
-------------------------------
66193.08129882812
41444.671875
Epoch 3
-------------------------------
65606.30029296875
40696.908854166664
Epoch 4
-------------------------------
65151.26513671875
39952.2421875
Epoch 5
-------------------------------
64470.614013671875
39148.011067708336
Epoch 6
-------------------------------
63942.088623046875
38271.747395833336
Epoch 7
-------------------------------
63314.432373046875
37435.427734375
Epoch 8
-------------------------------
62752.94775390625
36526.688151041664
Epoch 9
-------------------------------
62266.019287109375
35755.177734375
Epoch 10
-------------------------------
61470.832763671875
34859.613932291664
Epoch 11
-------------------------------
60958.57958984375
34013.835286458336
Epoch 12
-------------------------------
60460.94873046875
33192.184244791664
Epoch 13
-------------------------------
59777.9638671875
32414.892578125
Epoch 14
--

In [26]:
## metaselection example
n_features = meta_train.Z.shape[1]
_, n_forecasters, horizon  = meta_train.B.shape
window_x = meta_train.X.shape[2]

model = MetaNN.MetaSelection(n_features= n_features, n_forecasters=n_forecasters,window_x=window_x, Use_z= Use_z,n_known_features=n_known_features,horizon=horizon).to(device)
loss_fn = torch.nn.CrossEntropyLoss()
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(),  lr=learning_rate)
epochs = 50
meta_learning = meta_learning_run(model,loss_fn, optimizer, train_dataloader,test_dataloader, device, epochs)
loss, pred = meta_learning.forecast(test_dataloader)
preds.append(np.expand_dims(pred.cpu().numpy(),1))

Epoch 1
-------------------------------
1.6137756630778313
1.6130115985870361
Epoch 2
-------------------------------
1.6109770312905312
1.6098813613255818
Epoch 3
-------------------------------
1.6078073307871819
1.605297843615214
Epoch 4
-------------------------------
1.6047900393605232
1.6010055144627888
Epoch 5
-------------------------------
1.6019626408815384
1.5962287187576294
Epoch 6
-------------------------------
1.5984884798526764
1.5923510789871216
Epoch 7
-------------------------------
1.5948734506964684
1.587559660275777
Epoch 8
-------------------------------
1.5907770618796349
1.580918550491333
Epoch 9
-------------------------------
1.5863177627325058
1.5746070941289265
Epoch 10
-------------------------------
1.5818346589803696
1.5685499906539917
Epoch 11
-------------------------------
1.5765976756811142
1.5610446532567341
Epoch 12
-------------------------------
1.5715369209647179
1.5534095764160156
Epoch 13
-------------------------------
1.565579205751419
1.544

In [27]:
## metalossn example
n_features = meta_train.Z.shape[1]
_, n_forecasters, horizon  = meta_train.B.shape
window_x = meta_train.X.shape[2]

model = MetaNN.MetaLoss(n_features= n_features, n_forecasters=n_forecasters,window_x=window_x, Use_z= Use_z,n_known_features= n_known_features,horizon=horizon).to(device)
loss_fn = torch.nn.MSELoss()
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(),  lr=learning_rate)
epochs = 50
meta_learning = meta_learning_run(model,loss_fn, optimizer, train_dataloader,test_dataloader, device, epochs)
loss, pred = meta_learning.forecast(test_dataloader)
preds.append(np.expand_dims(pred.cpu().numpy(),1))

Epoch 1
-------------------------------
13918974464.0
7799354880.0
Epoch 2
-------------------------------
13917690560.0
7799352661.333333
Epoch 3
-------------------------------
13848876928.0
7799349760.0
Epoch 4
-------------------------------
13861092224.0
7799347029.333333
Epoch 5
-------------------------------
13880778816.0
7799344810.666667
Epoch 6
-------------------------------
13945496256.0
7799341568.0
Epoch 7
-------------------------------
13892759488.0
7799337984.0
Epoch 8
-------------------------------
13875955904.0
7799333546.666667
Epoch 9
-------------------------------
13866547968.0
7799328426.666667
Epoch 10
-------------------------------
13857211200.0
7799321941.333333
Epoch 11
-------------------------------
13803114304.0
7799312725.333333
Epoch 12
-------------------------------
13891948736.0
7799301802.666667
Epoch 13
-------------------------------
13870959296.0
7799289344.0
Epoch 14
-------------------------------
13847398336.0
7799277738.666667
Epoch 15
---

In [28]:
pd.DataFrame(meta_test.evaluate(np.concatenate(preds,1),utils.metrics),index= meta_test.forecasters + ['meta_comb','meta_select','meta_loss'])

,MAE,MSE,sMAPE1,sMAPE2,RMSSE
SimpleExpSmoothing,279.805108,113379.111133,0.401223,0.384955,1.149770
ExponentialSmoothing,283.127890,115406.332574,0.410289,0.393319,1.159131
lightgbm_direct,129.561539,28544.295530,0.179451,0.193225,0.567932
lightgbm_mimo,127.199703,27033.851736,0.181451,0.193018,0.559178
rf_mimo,137.996915,33158.710936,0.178129,0.196899,0.631642
meta_comb,123.118840,25262.495127,0.170102,0.185577,0.538353
meta_select,127.199703,27033.851649,0.181451,0.193018,0.559178
meta_loss,165.998662,43014.574643,0.228979,0.238942,0.717454
